In [11]:
import numpy as np

In [12]:
class Activation:

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def sigmoid_derivative(x):
        return Activation.sigmoid(x) * (1 - Activation.sigmoid(x))

In [13]:
# Neuron Layer
class Layer:
    def __init__(self, size, activation_function):
        self.size = size
        self.activation_function = activation_function
        self.output = None
        self.input = None
        self.weights = None
        self.bias = None

In [14]:
class Model:

    def __init__(self):
        self.layers = []
    def add(self, layer):
        self.layers.append(layer)

    def setup(self):
        for i in range(1, len(self.layers)):
            self.layers[i].weights = np.random.randn(self.layers[i-1].size, self.layers[i].size)
            self.layers[i].bias = np.random.randn(self.layers[i].size)

In [15]:
# Loss Function

class LossFunction:
    @staticmethod
    def mean_squared_error(y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    @staticmethod
    def mean_squared_error_derivative(y_true, y_pred):
        return 2 * (y_pred - y_true) / y_true.size

In [16]:
# Forward Propagation

class ForwardProp:
    @staticmethod
    def compute(model, input_data):
        for i in range(1, len(model.layers)):
            if i == 1:
                model.layers[i].input = input_data
            else:
                model.layers[i].input = model.layers[i-1].output
            z = np.dot(model.layers[i].input, model.layers[i].weights) + model.layers[i].bias
            model.layers[i].output = Activation.sigmoid(z)
        return model.layers[-1].output

In [17]:
# Backward Propagation

class BackProp:
    @staticmethod
    def compute(model, y_true, y_pred):
        changes = {}
        for i in reversed(range(1, len(model.layers))):
            if i == len(model.layers) - 1:
                error = LossFunction.mean_squared_error_derivative(y_true, y_pred)
                delta = error * Activation.sigmoid_derivative(y_pred)
            else:
                error = np.dot(delta, model.layers[i+1].weights.T)
                delta = error * Activation.sigmoid_derivative(model.layers[i].output)

            weight_change = np.dot(model.layers[i].input.T, delta)
            bias_change = np.sum(delta, axis=0, keepdims=True)
            changes[i] = (weight_change, bias_change)
        return changes

In [18]:
# Gradient Descent Optimization

class GradDescent:
    @staticmethod
    def update(model, changes, lr=0.1):
        for i in range(1, len(model.layers)):
            model.layers[i].weights -= lr * changes[i][0]
            model.layers[i].bias -= lr * changes[i][1].reshape(model.layers[i].bias.shape)

In [19]:
# Training Process

class Training:
    @staticmethod
    def train(model, x_train, y_train, epochs, lr):
        for epoch in range(epochs):
            # Forward propagation
            y_pred = ForwardProp.compute(model, x_train)
            # Backward propagation
            changes = BackProp.compute(model, y_train, y_pred)
            # Update weights
            GradDescent.update(model, changes, lr)
            # Print loss
            if epoch % 100 == 0:
                loss = LossFunction.mean_squared_error(y_train, y_pred)
                print(f'Epoch {epoch}, Loss: {loss}')

In [20]:
x_train = np.array([[0,0], [0,1], [1,0], [1,1]])
y_train = np.array([[0], [1], [1], [0]])


nn_model = Model()
nn_model.add(Layer(size=2, activation_function='sigmoid'))  # Input layer
nn_model.add(Layer(size=3, activation_function='sigmoid'))  # Hidden layer
nn_model.add(Layer(size=1, activation_function='sigmoid'))  # Output layer

nn_model.setup()

Training.train(nn_model, x_train, y_train, epochs=1000, lr=0.1)


y_pred = ForwardProp.compute(nn_model, x_train)

print(f'Predictions:\n{y_pred}')

Epoch 0, Loss: 0.4120178111497476
Epoch 100, Loss: 0.2514285790857201
Epoch 200, Loss: 0.25005947347564106
Epoch 300, Loss: 0.24976449057944783
Epoch 400, Loss: 0.2494857491859962
Epoch 500, Loss: 0.24921822832709997
Epoch 600, Loss: 0.24896071413069404
Epoch 700, Loss: 0.2487122058368037
Epoch 800, Loss: 0.24847183336441686
Epoch 900, Loss: 0.2482388381135382
Predictions:
[[0.47352703]
 [0.50534733]
 [0.50244692]
 [0.5249591 ]]
